
Certainly! The example you provided shows a structure for processing AVI files in selected directories, resizing them, and saving them in a new folder. To adapt this structure for your requirement of merging Excel (.xlsx) files into one file, with each original file name as the sheet name in the final Excel file, we'll need to make several adjustments and use pandas for handling Excel files, along with tkinter for the directory selection GUI.

Here's a modified version of your script for handling Excel files. This version assumes you have pandas installed (pip install pandas), which is a powerful library for data manipulation and analysis, especially suited for working with tabular data like Excel spreadsheets.

In [1]:
import os
import pandas as pd
import tkinter as tk
from tkinter import filedialog
import openpyxl

def select_directory():
    root = tk.Tk()
    root.withdraw()  # Hide the main window
    root.attributes('-topmost', True)  # Make the file dialog appear on top
    selected_path = filedialog.askdirectory()
    root.destroy()
    return selected_path

def collect_excel_files(folder_path, collected_data):
    for entry in os.listdir(folder_path):
        full_path = os.path.join(folder_path, entry)
        if os.path.isdir(full_path):
            collect_excel_files(full_path, collected_data)
        elif entry.endswith('.xlsx'):
            try:
                xls = pd.ExcelFile(full_path)
                if len(xls.sheet_names) > 1:
                    second_sheet_name = xls.sheet_names[1]
                    df = pd.read_excel(full_path, sheet_name=second_sheet_name)
                    if not df.empty:
                        # Remove the '.xlsx' part from the filename for the sheet name
                        base_filename = os.path.splitext(os.path.basename(full_path))[0]
                        # Remove '_additional_metrics' from the base_filename
                        base_filename = base_filename.replace('_additional_metrics', '')
                        unique_key = os.path.relpath(os.path.join(folder_path, base_filename), start=folder_path)
                        collected_data[unique_key] = df
                        print(f"Adding second sheet from file to merge: {full_path}")
                else:
                    print(f"No second sheet found in file: {full_path}")
            except Exception as e:
                print(f"Error processing file {entry}: {e}")

def write_collected_data_to_excel(collected_data, output_file_path):
    if os.path.exists(output_file_path):
        os.remove(output_file_path)  # Remove existing file before writing
        print(f"{output_file_path} already exists. It will be overwritten.")
    
    with pd.ExcelWriter(output_file_path, engine='openpyxl') as writer:
        for sheet_name, df in collected_data.items():
            # Remove '_additional_metrics' from the sheet name if it exists
            safe_sheet_name = sheet_name.replace('_additional_metrics', '')
            # Ensure the sheet name is within Excel's limit and does not include '.xlsx'
            safe_sheet_name = safe_sheet_name[:31]
            df.to_excel(writer, sheet_name=safe_sheet_name, index=False)
            print(f"Writing sheet: {safe_sheet_name}")
    
    print(f"All Excel files merged into {output_file_path}")

    # Auto-fit column widths for all sheets
    wb = openpyxl.load_workbook(output_file_path)
    for sheet_name in wb.sheetnames:
        ws = wb[sheet_name]
        for col in ws.columns:
            max_length = 0
            column = col[0].column_letter  # Get the column name
            for cell in col:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(str(cell.value))
                except:
                    pass
            adjusted_width = (max_length + 2)
            ws.column_dimensions[column].width = adjusted_width
    
    wb.save(output_file_path)
    print(f"Adjusted column widths and saved the workbook at {output_file_path}")

if __name__ == "__main__":
    base_path = select_directory()
    if not base_path:
        print("No folder selected.")
    else:
        output_file_path = os.path.join(base_path, "merged_excel.xlsx")
        collected_data = {}
        collect_excel_files(base_path, collected_data)
        if collected_data:
            write_collected_data_to_excel(collected_data, output_file_path)
        else:
            print("No suitable Excel files found to merge.")




Adding second sheet from file to merge: E:/Particle tracking/Summer Semester Jun-Aug 2024/14JUL24/Cycle study/0 cycles/150 mVpp/Compilation\Compiled xlsx\0 cycles_additional_metrics.xlsx
Adding second sheet from file to merge: E:/Particle tracking/Summer Semester Jun-Aug 2024/14JUL24/Cycle study/0 cycles/150 mVpp/Compilation\Compiled xlsx\0 cycles_distance_and_speed_metrics.xlsx
Adding second sheet from file to merge: E:/Particle tracking/Summer Semester Jun-Aug 2024/14JUL24/Cycle study/0 cycles/150 mVpp/Compilation\Compiled xlsx\24 cycles_additional_metrics.xlsx
Adding second sheet from file to merge: E:/Particle tracking/Summer Semester Jun-Aug 2024/14JUL24/Cycle study/0 cycles/150 mVpp/Compilation\Compiled xlsx\24 cycles_distance_and_speed_metrics.xlsx
Adding second sheet from file to merge: E:/Particle tracking/Summer Semester Jun-Aug 2024/14JUL24/Cycle study/0 cycles/150 mVpp/Compilation\Compiled xlsx\240 cycles_additional_metrics.xlsx
Adding second sheet from file to merge: E:/Pa

In [4]:
import os
import pandas as pd
import tkinter as tk
from tkinter import filedialog
import openpyxl

def select_directory():
    root = tk.Tk()
    root.withdraw()
    root.attributes('-topmost', True)
    selected_path = filedialog.askdirectory()
    root.destroy()
    return selected_path

def collect_excel_files(folder_path, collected_data):
    for entry in os.listdir(folder_path):
        full_path = os.path.join(folder_path, entry)
        if os.path.isdir(full_path):
            collect_excel_files(full_path, collected_data)
        elif entry.endswith('.xlsx') and '_additional_metrics' in entry:
            try:
                xls = pd.ExcelFile(full_path)
                if len(xls.sheet_names) > 1:
                    second_sheet_name = xls.sheet_names[1]
                    df = pd.read_excel(full_path, sheet_name=second_sheet_name)
                    if not df.empty:
                        base_filename = os.path.splitext(os.path.basename(full_path))[0]
                        base_filename = base_filename.replace('_additional_metrics', '')
                        unique_key = os.path.relpath(os.path.join(folder_path, base_filename), start=folder_path)
                        collected_data[unique_key] = df
                        print(f"Adding second sheet from file to merge: {full_path}")
                else:
                    print(f"No second sheet found in file: {full_path}")
            except Exception as e:
                print(f"Error processing file {entry}: {e}")

def write_collected_data_to_excel(collected_data, output_file_path):
    if os.path.exists(output_file_path):
        os.remove(output_file_path)
        print(f"{output_file_path} already exists. It will be overwritten.")
    
    with pd.ExcelWriter(output_file_path, engine='openpyxl') as writer:
        for sheet_name, df in collected_data.items():
            safe_sheet_name = sheet_name.replace('_additional_metrics', '')
            safe_sheet_name = safe_sheet_name[:31]
            df.to_excel(writer, sheet_name=safe_sheet_name, index=False)
            print(f"Writing sheet: {safe_sheet_name}")
    
    print(f"All Excel files merged into {output_file_path}")
    
    wb = openpyxl.load_workbook(output_file_path)
    for sheet_name in wb.sheetnames:
        ws = wb[sheet_name]
        for col in ws.columns:
            max_length = 0
            column = col[0].column_letter
            for cell in col:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(str(cell.value))
                except:
                    pass
            adjusted_width = (max_length + 2)
            ws.column_dimensions[column].width = adjusted_width
    
    wb.save(output_file_path)
    print(f"Adjusted column widths and saved the workbook at {output_file_path}")

if __name__ == "__main__":
    base_path = select_directory()
    if not base_path:
        print("No folder selected.")
    else:
        output_file_path = os.path.join(base_path, "merged_excel.xlsx")
        collected_data = {}
        collect_excel_files(base_path, collected_data)
        if collected_data:
            write_collected_data_to_excel(collected_data, output_file_path)
        else:
            print("No suitable Excel files found to merge.")

Adding second sheet from file to merge: E:/Particle tracking/Summer Semester Jun-Aug 2024/08JUL24/DOPC HMSM/1\Compilation\Compiled xlsx\0 mVpp - 1_additional_metrics.xlsx
Adding second sheet from file to merge: E:/Particle tracking/Summer Semester Jun-Aug 2024/08JUL24/DOPC HMSM/1\Compilation\Compiled xlsx\100 mVpp - 1_additional_metrics.xlsx
Adding second sheet from file to merge: E:/Particle tracking/Summer Semester Jun-Aug 2024/08JUL24/DOPC HMSM/1\Compilation\Compiled xlsx\125 mVpp - 1_additional_metrics.xlsx
Adding second sheet from file to merge: E:/Particle tracking/Summer Semester Jun-Aug 2024/08JUL24/DOPC HMSM/1\Compilation\Compiled xlsx\150 mVpp - 1_additional_metrics.xlsx
Adding second sheet from file to merge: E:/Particle tracking/Summer Semester Jun-Aug 2024/08JUL24/DOPC HMSM/1\Compilation\Compiled xlsx\175 mVpp - 1_additional_metrics.xlsx
Adding second sheet from file to merge: E:/Particle tracking/Summer Semester Jun-Aug 2024/08JUL24/DOPC HMSM/1\Compilation\Compiled xlsx\2